Using SX generate parquet files for testing with join infrastructure. Also, using new SX frontend.

Constants:

In [1]:
# A Z to ee sample - Release 21
ds_name = r"mc16_13TeV:mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.DAOD_PHYS.e3601_e5984_s3126_r10201_r10210_p5313"

What we'll need for imports

In [2]:
from servicex_client.servicex_client import ServiceXClient
from servicex_client.dataset_identifier import RucioDatasetIdentifier
from servicex_client.models import ResultFormat
from func_adl_servicex_xaodr22.event_collection import Event

# import logging
# logging.basicConfig(level=logging.DEBUG)

In [3]:
sx = ServiceXClient(backend="test4")
did = RucioDatasetIdentifier(ds_name, num_files=10)
ds_raw = sx.func_adl_dataset(
    did, codegen="atlasr21", title="Zee", result_format=ResultFormat.parquet, item_type=Event
)

from func_adl_servicex_xaodr22 import calib_tools
#ds = calib_tools.apply_calibration(ds_raw, "PHYS")
ds = calib_tools.query_update(ds_raw, calib_tools.default_config("PHYSLITE"))

In [4]:
electrons = (
    ds.Select(lambda e: e.Electrons('Electrons'))
    .Select(
        lambda e: e.Where(lambda e: (e.pt() / 1000 > 25.0) and (abs(e.eta()) < 2.5))
    )
    .Select(lambda e: e.Select(lambda e: e.pt() / 1000))
).as_files() #.generate_selection_string()

Output()

ServiceX Transform bb4f93bf-5946-4654-b839-23a72c122be8

Transforms completed successfully

TransformedResults(hash='311aecb8f5ce1a2046732df94f42b27909b8c112a53fbddb7e3fb9e7435eaecb', title='Zee', codegen='atlasr21', request_id='bb4f93bf-5946-4654-b839-23a72c122be8', submit_time=datetime.datetime(2023, 7, 13, 7, 20, 37, 594651, tzinfo=datetime.timezone.utc), data_dir='sx_cache/bb4f93bf-5946-4654-b839-23a72c122be8', file_list=['C:/Users/gordo/Code/iris-hep/joiner/sx_cache/bb4f93bf-5946-4654-b839-23a72c122be8/_fea60efda9349ddcf67332e74684c32fb0e1279e1.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/bb4f93bf-5946-4654-b839-23a72c122be8/_5ea15726988e2cabe77ef8ff3060e301abb4adad3.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/bb4f93bf-5946-4654-b839-23a72c122be8/_300acfa59326ff8ec94539acc44013a581d700593.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/bb4f93bf-5946-4654-b839-23a72c122be8/_62d32c51d78d6612ce0755ea4d7db714e5bd61ac5.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/bb4f93bf-5946-4654-b839-23a72c12

In [8]:
import awkward as ak
pts = ak.from_parquet(electrons.file_list)
pts

<Array [{col1: []}, ... {col1: [49.5]}] type='460000 * {"col1": var * float64}'>

# Query 1 - Electrons

Good healthy electrons that should be part of a Z mass peak: